In [ ]:
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF
from scipy.spatial.distance import pdist, cdist, squareform
import numpy as np
import tensorflow as tf
import gpflow


In [ ]:
X, y = make_friedman2(n_samples=500, noise=0, random_state=0)
kernel = RBF() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
         random_state=0).fit(X, y)
gpr.score(X, y)

In [ ]:
.reshape(-1, 1)

In [ ]:
y = y.reshape(-1, 1)
k = gpflow.kernels.Matern52()
m = gpflow.models.GPR(data=(X, y), kernel=k, mean_function=None)
opt = gpflow.optimizers.Scipy()
def objective_closure():
    return - m.log_marginal_likelihood()

opt_logs = opt.minimize(objective_closure,
                        m.trainable_variables,
                        options=dict(maxiter=20))



In [ ]:
m.predict_f(X)

In [ ]:
print(gpr.kernel_(X).shape, gpr.kernel_)#K = self.kernel_(self.X_train_)
print (X.shape, gpr.X_train_.shape)

dists = pdist(X / 1, metric='sqeuclidean')
K = np.exp(-.5 * dists)
print(K.shape)
# convert from upper-triangular matrix to square matrix
K = squareform(K)
print(K.shape)
np.fill_diagonal(K, 1)


In [ ]:
t = tf.cast(tf.range(4)*2, tf.float64)
t_dist = tf.expand_dims(t, axis=0) - tf.expand_dims(t, axis=1)
t_ = tf.transpose(tf.reshape(tf.tile(t, [4]), [ 4, tf.shape(t)[0]]))
t_prime = tf.reshape(tf.tile(t, [4]), [ 4, tf.shape(t)[0]])

D = tf.ones(4)

print(t)
print(t_dist)
print(t_)
print(t_prime-t_)

m = [1,2,3,4]
# Compute m[i] * (t'-t) + t' for all i, t, t'
result = np.zeros((4, 4))
for i, t_ in enumerate(t):
    for j, t_prime in enumerate(t):
        for mk in m:
            result[i, j] += mk * (t_prime - t_) + t_prime
    
print('Result 1:')
print(result)
print()
print('Result 2:')

add = tf.transpose(tf.reshape(tf.tile(t, [4]), [ 4, tf.shape(t)[0]]))
result = np.zeros((4, 4))
result += m*t_dist + add
print(result)

In [ ]:
import numpy as np
t = 3
tprime = 3
l = 2
np.exp(-((t-tprime)**2)/(l**2))

times   =   np.array([2.0,4.0, 6.0, 8.0])[:,None]
times.shape
times[2]

In [ ]:
num_times=3
num_genes=2
from gpflow.utilities import print_summary, positive
from tensorflow_probability import bijectors as tfb
from tensorflow import math as tm
import math
PI = tf.constant(math.pi, dtype='float64')

class Kern(gpflow.kernels.Kernel):
    
    def __init__(self):
        super().__init__(active_dims=[0])
        self.lengthscale = gpflow.Parameter(1.0, transform=positive())
#         B = tf.ones(5)
#         self.B = gpflow.Parameter(B)
#         self.D = gpflow.Parameter(np.random.uniform(0.5, 1, 5), transform=positive())
#         S = tf.ones(5)
        affine = tfb.AffineScalar(shift=tf.cast(0., tf.float64),
                                  scale=tf.cast(3.-0., tf.float64))
        sigmoid = tfb.Sigmoid()
        logistic = tfb.Chain([affine, sigmoid])

        self.D = [gpflow.Parameter(0.7, transform=logistic, dtype=tf.float64) for _ in range(num_genes)]
        self.D[0].trainable = False
        self.D[0].assign(0.8)

        self.S = [gpflow.Parameter(0.7, transform=logistic, dtype=tf.float64) for _ in range(num_genes)]
        self.S[0].trainable = False
        self.S[0].assign(1)

    def K(self, X, X2=None):
        block_size = num_times
        if X2 is None:
            shape = [X.shape[0],X.shape[0]]
            K_xx = tf.zeros(shape, dtype='float64')
            for j in range(num_genes):
                for k in range(num_genes):
                    mask = np.ones(shape)
                    other = np.zeros(shape)
                    mask[j*block_size:(j+1)*block_size, 
                         k*block_size:(k+1)*block_size] = 0

                    pad_top = j*block_size
                    pad_left = k*block_size
                    pad_right = 0 if k == num_genes-1 else shape[0]-block_size-pad_left
                    pad_bottom = 0 if j == num_genes-1 else shape[0]-block_size-pad_top
                    other = tf.pad(self.k_xx(j, k),
                                   tf.constant([
                                       [pad_top,pad_bottom],
                                       [pad_left,pad_right]
                                   ]), 'CONSTANT'
                                  )
    #                     print(j, k, pad_right, pad_bottom, other.shape)
                    K_xx = K_xx * mask + other * (1 - mask)


            return K_xx
        else:
            print('K not none K_xf\n')
            shape = [X.shape[0],num_times]
            K_xf = tf.zeros(shape, dtype='float64')
            for j in range(num_genes):
                mask = np.ones(shape)
                other = np.zeros(shape)
                mask[j*block_size:(j+1)*block_size] = 0
                other[j*block_size:(j+1)*block_size] = self.k_xf(j, X)

                K_xf = K_xf * mask + other * (1-mask) 
            return K_xf


    def gamma(self, k):
        return self.D[k]*self.lengthscale/2

    def h(self, k, j, tprime, t):
        l = self.lengthscale

        multiplier = tm.exp(self.gamma(k))**2 / (self.D[j]+self.D[k])
        first_erf_term = tm.erf((tprime-t)/l - self.gamma(k)) + tm.erf(t/l + self.gamma(k))
        second_erf_term = tm.erf(tprime/l - self.gamma(k)) + tm.erf(self.gamma(k))
        return multiplier * (tm.exp(-self.D[k]*(tprime-t)) * first_erf_term - \
                             tm.exp(-self.D[k]*tprime-self.D[j]) * second_erf_term)


    def h_quick(self, k, j, primefirst=True):
        l = self.lengthscale
        t_prime, t_, t_dist = self.get_distance_matrix(primefirst=primefirst, size=num_times)
            
        multiplier = tm.exp(self.gamma(k))**2 / (self.D[j]+self.D[k])
        first_erf_term = tm.erf(t_dist/l - self.gamma(k)) + tm.erf(t_/l + self.gamma(k))
        second_erf_term = tm.erf(t_prime/l - self.gamma(k)) + tm.erf(self.gamma(k))
        
        return multiplier * (tf.multiply(tm.exp(-tm.multiply(self.D[k],t_dist)) , first_erf_term) - \
                             tf.multiply(tm.exp(-tm.multiply(self.D[k],t_prime)-self.D[j]) , second_erf_term))
    

    def k_xx(self, j, k):
        '''k_xx(t, tprime)'''
        mult = self.S[j]*self.S[k]*self.lengthscale*0.5*tm.sqrt(PI)
        return mult*(self.h_quick(k, j) + self.h_quick(j, k, primefirst=False))

    def k_xx_(self, j, k):
        '''k_xx(t, tprime)'''
        k_xx = np.zeros((num_times, num_times))
        for tprime in range(num_times):
            for t in range(num_times):
                mult = self.S[j]*self.S[k]*self.lengthscale*0.5*tm.sqrt(PI)
                k_xx[t,tprime] = mult*(self.h(k, j, tprime*2, t*2) + self.h(j, k, t*2, tprime*2))
        print(k_xx)
        return k_xx

    def get_distance_matrix(self, primefirst=True, size=7):
        t = tf.cast(tf.range(size)*2, tf.float64)
        t_ = tf.transpose(tf.reshape(tf.tile(t, [size]), [ size, tf.shape(t)[0]]))
        t_prime = tf.reshape(tf.tile(t, [size]), [ size, tf.shape(t)[0]])
        if not primefirst:
            t_prime = tf.transpose(tf.reshape(tf.tile(t, [size]), [ size, tf.shape(t)[0]]))
            t_ = tf.reshape(tf.tile(t, [size]), [ size, tf.shape(t)[0]])

        return t_prime, t_, t_prime-t_


In [ ]:
X = np.arange(num_times, dtype='float64')*2
X = np.c_[[X for _ in range(num_genes)]].reshape(-1)
print(X)
k = Kern()
display(k.K(X))

In [ ]:
array([[0.        , 0.19252085, 0.03952372, 0.        , 0.15377981, 0.03846892],
       [0.19252085, 1.09337319, 0.3065648 , 0.13141296, 0.78650799, 0.24865445],
       [0.03952372, 0.3065648 , 1.34475164, 0.02697853, 0.21804878, 0.98498977],
       [0.        , 0.13141296, 0.02697853, 0.        , 0.10543729, 0.02637576],
       [0.15377981, 0.78650799, 0.21804878, 0.10543729, 0.56618178, 0.17712899],
       [0.03846892, 0.24865445, 0.98498977, 0.02637576, 0.17712899, 0.72327064]])>
6, 6